In [1]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [2]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [3]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [4]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [5]:
# !fc-cache -f

In [6]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [7]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [250]:
SAVE = True
LOCAL = True
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [251]:
%%capture pwd
!pwd

In [252]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('the-phoenix-that-never-rises-whats-holding-back-pakistans-economy', 300, 500)

In [253]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [254]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [255]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                            if c in i['encoding'][x]['axis']:
                                i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [256]:
df = pd.read_csv("raw/Default Dataset.csv",header=None)
df.columns=['c','v']
columns=df.sort_values(by='v')['c'].values

In [257]:
readme, f, fc = save(df,"fig1_prod",LOCAL)

,c,v
0,China,8.117113
1,Myanmar,7.285209
2,India,4.716810
3,Vietnam,4.127208
4,Laos,4.119132


In [258]:
xmin=9
xmax=9
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':'India'},{'x':xmax,'y':'China'}])).mark_bar(color=colors["eco-gray"],opacity=0.01,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:N",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='v:Q')
bar1=base.mark_bar(color=colors['eco-light-blue'],height=8).encode(y=alt.Y('c:N',sort='x',scale=alt.Scale(domain=columns)))
bar2=base.mark_bar(color=colors['eco-dot']).encode(y=alt.Y('c:N',sort='x')).transform_filter('datum.c=="Pakistan"')

layer1 = (
    (xaxis+bar1+bar2).properties(height=500, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 'Kenya', "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:Q',y='y:N',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig1_prod_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 2

In [259]:
df = pd.read_csv("raw/Default Dataset2.csv",header=None)
df.columns=['c1','v','c2']
df['c']=df['c1']+'#'+df['c2']
columns=df['c'].values

In [260]:
readme, f, fc = save(df,"fig2_ind",LOCAL)

,c1,v,c2,c
0,Mining,-5.867769,Pakistan,Mining#Pakistan
1,Utilities,-4.338843,Pakistan,Utilities#Pakistan
2,Transport,-1.280992,Pakistan,Transport#Pakistan
3,Real Estate,-0.578512,Pakistan,Real Estate#Pakistan
4,Construction,-0.165289,Pakistan,Construction#Pakistan


In [261]:
xmin=14
xmax=14
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':'Mining#Pakasitan'},{'x':xmax,'y':'Other#China'}])).mark_bar(color=colors["eco-gray"],opacity=0.01,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:N",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='v:Q')
bar1=base.mark_bar(color=colors['eco-light-blue'],height=8).encode(y=alt.Y('c:N',sort='x',scale=alt.Scale(domain=columns),
    axis=alt.Axis(labelExpr='split(datum.label,"#")[0]')),
    color=alt.Color('c2:N',legend=alt.Legend(orient='top-right',title='',labelColor=colors['eco-gray'], labelOpacity=mo+0.2,
            labelFontSize=12),scale=alt.Scale(range=[colors['eco-dot'],colors['eco-turquiose'],colors['eco-light-blue'],colors['eco-green']]))
)

layer1 = (
    (xaxis+bar1).properties(height=500, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 'Mining#Pakistan', "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:Q',y='y:N',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig2_ind_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 3

In [262]:
df = pd.read_stata("raw/avg_gvc_data_sector.dta")
df=df[['isocode3','lnavg_share_trade00','avg_gvc_part_exp_int','export_intensity']].dropna()

In [263]:
readme, f, fc = save(df,"fig3_trade",LOCAL)

,isocode3,lnavg_share_trade00,avg_gvc_part_exp_int,export_intensity
1,AUS,3.808008,0.304734,0.0
2,AUS,3.808008,0.420851,1.0
3,AUT,4.645868,0.514516,0.0
4,AUT,4.645868,0.623465,1.0
5,BGD,2.761400,0.266212,0.0


In [264]:
xmin=2
xmax=6
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="log ( Trade openness )",
            titleY=-15,
            titleX=337,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Participation rate",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.1f',
            tickCount=8
        ),
        scale=alt.Scale(domain=[0.1, 0.9]),
    )
)

base = alt.Chart(fc).encode(x='lnavg_share_trade00:Q',y='avg_gvc_part_exp_int:Q')
points1=base.mark_point(color=colors['eco-light-blue'],fill=colors['eco-light-blue']).transform_filter('datum.export_intensity==0')
points2=base.mark_point(color=colors['eco-dot'],fill=colors['eco-dot']).transform_filter('datum.export_intensity==1')
labels1=points1.mark_text(color=colors['eco-light-blue'],text='PAK',fontSize=12,dx=18).transform_filter('datum.isocode3=="PAK"')
labels2=points2.mark_text(color=colors['eco-dot'],text='PAK',fontSize=12,dx=18).transform_filter('datum.isocode3=="PAK"')
reg=base.mark_line(color=colors['eco-gray'],opacity=0.3).transform_regression(
        "lnavg_share_trade00", "avg_gvc_part_exp_int", method="linear", as_=["x","y"]
    ).encode(x='x:Q',y='y:Q')
labels3=alt.Chart(pd.DataFrame([{'x':2.1,'y':.82,'t':'Export sector'},{'x':2.1,'y':.77,'t':'Non-export sector'}])).mark_text(
    fontSize=12,align='left').encode(x='x:Q',y='y:Q',
    text='t:N',color=alt.Color('t:N',legend=None,scale=alt.Scale(range=[colors['eco-dot'],colors['eco-light-blue']]))
                       )
layer1 = (
    (xaxis+reg+points1+points2+labels1+labels2+labels3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 0.9, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig3_trade_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 4

In [265]:
df = pd.read_excel("raw/Fig4.xls",sheet_name='Sheet1')
df.columns=['t','d1','d2']

In [266]:
readme, f, fc = save(df,"fig4_debt",LOCAL)

,t,d1,d2
0,2016-11-30,-6670.19,6.67019
1,2016-12-31,-8642.18,8.64218
2,2017-01-31,-8114.14,8.11414
3,2017-02-28,-8029.27,8.02927
4,2017-03-31,-8165.76,8.16576


In [267]:
xmin='2016-11-01'
xmax='2024-01-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="$ billion",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='t:T')
line1=base.mark_line(color=colors['eco-green'],clip=True,interpolate='basis',size=3).encode(y='d2:Q')

layer1 = (
    (xaxis+line1).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 35, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig4_debt_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 5

In [268]:
df = pd.read_excel('raw/Fig5.xlsx')
df['Generation Capacity']=df['Generation Capacity'].interpolate()
df=df.dropna()

In [269]:
readme, f, fc = save(df,"fig5_elec",LOCAL)

,Dataset,Series Key,Series,Observation Date,Observation Value,Unit,Observation Status,Days in months,Electricity Demand (SBP),Generation Capacity
59,Generation of Electricity by Sector,TS_GP_RLS_ELECGEN_M.E_001000,Total Electricity Generation,2017-06-30,11458.230,GWh,Normal,30,15914.208333,19020.0
60,Generation of Electricity by Sector,TS_GP_RLS_ELECGEN_M.E_001000,Total Electricity Generation,2017-07-31,12496.640,GWh,Normal,31,16796.559140,19415.5
61,Generation of Electricity by Sector,TS_GP_RLS_ELECGEN_M.E_001000,Total Electricity Generation,2017-08-31,12754.283,GWh,Normal,31,17142.853495,19811.0
62,Generation of Electricity by Sector,TS_GP_RLS_ELECGEN_M.E_001000,Total Electricity Generation,2017-09-30,11489.020,GWh,Normal,30,15956.972222,20206.5
63,Generation of Electricity by Sector,TS_GP_RLS_ELECGEN_M.E_001000,Total Electricity Generation,2017-10-31,10017.290,GWh,Normal,31,13464.099462,20602.0


In [270]:
xmin='2017-07-01'
xmax='2023-12-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="MW",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='Observation Date:T')
line1=base.mark_line(color=colors['eco-dot'],clip=True,interpolate='basis',size=3).encode(y='Electricity Demand (SBP):Q')
line2=base.mark_line(color=colors['eco-green'],clip=True,interpolate='basis',size=3).encode(y='Generation Capacity:Q')
area=base.mark_area(clip=True,interpolate='basis',size=3,opacity=0.5,
                   
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='white', offset=0),
               alt.GradientStop(color=colors['eco-green'], offset=1)],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0
    )
                   ).encode(y='Electricity Demand (SBP):Q',y2='Generation Capacity:Q')
text1=line1.mark_text(text='Electricity Demand (SBP)',align='left',color=colors['eco-dot']).transform_filter('datum["Observation Date"]>toDate("2023-12-01")')
text2=line2.mark_text(text='Max Generation Capacity',align='left',color=colors['eco-green']).transform_filter('datum["Observation Date"]>toDate("2023-12-01")')
text3=line2.mark_text(text='Surplus',align='left',color=colors['eco-green'],dy=70).transform_filter('datum["Observation Date"]>toDate("2023-12-01")')

layer1 = (
    (xaxis+line1+line2+area+text1+text2+text3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 35000, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig5_elec_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 6

In [271]:
df = pd.read_csv("raw/wpd_datasets.csv",skiprows=1)

In [272]:
readme, f, fc = save(df,"fig6_lab",LOCAL)

,x1,y1,x2,y2,c2
0,1.263026,0.153291,4.752499,0.719427,USA
1,1.384745,0.153985,4.451690,0.672611,DEU
2,1.539660,0.138715,4.417420,0.439490,GBR
3,1.639248,0.249079,3.221799,0.164331,CHN
4,1.689042,0.123537,2.734412,0.158599,PAK


In [273]:
xmin=1
xmax=5.5
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="log ( Labour productivity )",
            titleY=-15,
            titleX=337,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Productivity gap",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.1f',
            tickCount=8
        ),
        scale=alt.Scale(domain=[-0.1, 1.2]),
    )
)

base = alt.Chart(fc).encode(x='x1:Q',y='y1:Q')
points1=base.mark_point(color=colors['eco-light-blue'],fill=colors['eco-light-blue'])
points2=base.mark_point(color=colors['eco-dot'],fill=colors['eco-dot']).encode(x='x2:Q',y='y2:Q').transform_filter('datum.c2!="PAK"')
points3=base.mark_point(color=colors['eco-green'],fill=colors['eco-green']).encode(x='x2:Q',y='y2:Q').transform_filter('datum.c2=="PAK"')
labels1=points3.mark_text(color=colors['eco-green'],fontSize=12,dx=18,dy=-3).encode(text='c2:N')
labels2=points2.mark_text(color=colors['eco-dot'],fontSize=12,dx=18).encode(text='c2:N')
reg=base.mark_line(color=colors['eco-gray'],opacity=0.3).transform_regression(
        "x1", "y1", method="quad", as_=["x","y"]
    ).encode(x='x:Q',y='y:Q')
# labels3=alt.Chart(pd.DataFrame([{'x':2.1,'y':.82,'t':'Export sector'},{'x':2.1,'y':.77,'t':'Non-export sector'}])).mark_text(
#     fontSize=12,align='left').encode(x='x:Q',y='y:Q',
#     text='t:N',color=alt.Color('t:N',legend=None,scale=alt.Scale(range=[colors['eco-dot'],colors['eco-light-blue']]))
                       # )
layer1 = (
    (xaxis+reg+points1+points2+points3+labels1+labels2).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6_lab_local



alt.LayerChart(...)

alt.LayerChart(...)

# Post-process

## Clean up `SVG`s
`base64 URI` encode images

In [274]:
from base64 import b64encode

In [275]:
from os import listdir
from os.path import isfile, join
mypath='./visualisation/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
svgs=[i for i in onlyfiles if i[-4:]=='.svg']

In [276]:
for svg in svgs:
    s=open(mypath+svg,'r').read()
    si=s.find('"image mark" xlink:href=')
    if si>-1:
        s2=s[si+25:]
        imgurl=s2[:s2.find('"')]
        if 'data:image/png;' not in imgurl:
            s1=s[:si+25]+s2.replace(imgurl,'data:image/png;base64,'+b64encode(requests.get(imgurl).content).decode('utf-8'))
            open(mypath+svg,'w').write(s1)
            print(svg)

fig1_prod_local.svg
fig1_prod_local_dark.svg
fig2_ind_local.svg
fig2_ind_local_dark.svg
fig3_trade_local.svg
fig3_trade_local_dark.svg
fig4_debt_local.svg
fig4_debt_local_dark.svg
fig5_elec_local.svg
fig5_elec_local_dark.svg
fig6_lab_local.svg
fig6_lab_local_dark.svg


## README

In [277]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

## Figure 1  

Data: [`csv`](data/fig1_prod_local.csv)  
GitHub: [fig1_prod_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/the-phoenix-that-never-rises-whats-holding-back-pakistans-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_prod_local_local.png) [`svg`](visualisation/fig1_prod_local_local.svg) [`json`](visualisation/fig1_prod_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_prod_local.png) [`svg`](visualisation/fig1_prod_local.svg) [`json`](visualisation/fig1_prod_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_prod_local_local_no_branding.png) [`svg`](visualisation/fig1_prod_local_local_no_branding.svg) [`json`](visualisation/fig1_prod_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_prod_local_no_branding.png) [`svg`](visualisation/fig1_prod_local_no_branding.svg) [`json`](visualisation/fig1_prod_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_prod_local_local_dark.png) [`svg`](visualisation/fig1_prod_local_local_dark.svg) [`json`](visualisation/fig1_prod_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_prod_local_dark.png) [`svg`](visualisation/fig1_prod_local_dark.svg) [`json`](visualisation/fig1_prod_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_prod_local_local_no_branding_dark.png) [`svg`](visualisation/fig1_prod_local_local_no_branding_dark.svg) [`json`](visualisation/fig1_prod_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_prod_local_no_branding_dark.png) [`svg`](visualisation/fig1_prod_local_no_branding_dark.svg) [`json`](visualisation/fig1_prod_local_no_branding_dark.json)   

!["fig1_prod_local"](visualisation/fig1_prod_local.svg "fig1_prod_local")

  

!["fig1_prod_local_dark"](visualisation/fig1_prod_local_dark.svg "fig1_prod_local")

## Figure 2  

Data: [`csv`](data/fig2_ind_local.csv)  
GitHub: [fig2_ind_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/the-phoenix-that-never-rises-whats-holding-back-pakistans-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_ind_local_local.png) [`svg`](visualisation/fig2_ind_local_local.svg) [`json`](visualisation/fig2_ind_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_ind_local.png) [`svg`](visualisation/fig2_ind_local.svg) [`json`](visualisation/fig2_ind_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_ind_local_local_no_branding.png) [`svg`](visualisation/fig2_ind_local_local_no_branding.svg) [`json`](visualisation/fig2_ind_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_ind_local_no_branding.png) [`svg`](visualisation/fig2_ind_local_no_branding.svg) [`json`](visualisation/fig2_ind_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_ind_local_local_dark.png) [`svg`](visualisation/fig2_ind_local_local_dark.svg) [`json`](visualisation/fig2_ind_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_ind_local_dark.png) [`svg`](visualisation/fig2_ind_local_dark.svg) [`json`](visualisation/fig2_ind_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_ind_local_local_no_branding_dark.png) [`svg`](visualisation/fig2_ind_local_local_no_branding_dark.svg) [`json`](visualisation/fig2_ind_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_ind_local_no_branding_dark.png) [`svg`](visualisation/fig2_ind_local_no_branding_dark.svg) [`json`](visualisation/fig2_ind_local_no_branding_dark.json)   

!["fig2_ind_local"](visualisation/fig2_ind_local.svg "fig2_ind_local")

  

!["fig2_ind_local_dark"](visualisation/fig2_ind_local_dark.svg "fig2_ind_local")

## Figure 3  

Data: [`csv`](data/fig3_trade_local.csv)  
GitHub: [fig3_trade_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/the-phoenix-that-never-rises-whats-holding-back-pakistans-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_trade_local_local.png) [`svg`](visualisation/fig3_trade_local_local.svg) [`json`](visualisation/fig3_trade_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_trade_local.png) [`svg`](visualisation/fig3_trade_local.svg) [`json`](visualisation/fig3_trade_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_trade_local_local_no_branding.png) [`svg`](visualisation/fig3_trade_local_local_no_branding.svg) [`json`](visualisation/fig3_trade_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_trade_local_no_branding.png) [`svg`](visualisation/fig3_trade_local_no_branding.svg) [`json`](visualisation/fig3_trade_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_trade_local_local_dark.png) [`svg`](visualisation/fig3_trade_local_local_dark.svg) [`json`](visualisation/fig3_trade_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_trade_local_dark.png) [`svg`](visualisation/fig3_trade_local_dark.svg) [`json`](visualisation/fig3_trade_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_trade_local_local_no_branding_dark.png) [`svg`](visualisation/fig3_trade_local_local_no_branding_dark.svg) [`json`](visualisation/fig3_trade_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_trade_local_no_branding_dark.png) [`svg`](visualisation/fig3_trade_local_no_branding_dark.svg) [`json`](visualisation/fig3_trade_local_no_branding_dark.json)   

!["fig3_trade_local"](visualisation/fig3_trade_local.svg "fig3_trade_local")

  

!["fig3_trade_local_dark"](visualisation/fig3_trade_local_dark.svg "fig3_trade_local")

## Figure 4  

Data: [`csv`](data/fig4_debt_local.csv)  
GitHub: [fig4_debt_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/the-phoenix-that-never-rises-whats-holding-back-pakistans-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_debt_local_local.png) [`svg`](visualisation/fig4_debt_local_local.svg) [`json`](visualisation/fig4_debt_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_debt_local.png) [`svg`](visualisation/fig4_debt_local.svg) [`json`](visualisation/fig4_debt_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_debt_local_local_no_branding.png) [`svg`](visualisation/fig4_debt_local_local_no_branding.svg) [`json`](visualisation/fig4_debt_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_debt_local_no_branding.png) [`svg`](visualisation/fig4_debt_local_no_branding.svg) [`json`](visualisation/fig4_debt_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_debt_local_local_dark.png) [`svg`](visualisation/fig4_debt_local_local_dark.svg) [`json`](visualisation/fig4_debt_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_debt_local_dark.png) [`svg`](visualisation/fig4_debt_local_dark.svg) [`json`](visualisation/fig4_debt_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_debt_local_local_no_branding_dark.png) [`svg`](visualisation/fig4_debt_local_local_no_branding_dark.svg) [`json`](visualisation/fig4_debt_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_debt_local_no_branding_dark.png) [`svg`](visualisation/fig4_debt_local_no_branding_dark.svg) [`json`](visualisation/fig4_debt_local_no_branding_dark.json)   

!["fig4_debt_local"](visualisation/fig4_debt_local.svg "fig4_debt_local")

  

!["fig4_debt_local_dark"](visualisation/fig4_debt_local_dark.svg "fig4_debt_local")

## Figure 5  

Data: [`csv`](data/fig5_elec_local.csv)  
GitHub: [fig5_elec_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/the-phoenix-that-never-rises-whats-holding-back-pakistans-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_elec_local_local.png) [`svg`](visualisation/fig5_elec_local_local.svg) [`json`](visualisation/fig5_elec_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_elec_local.png) [`svg`](visualisation/fig5_elec_local.svg) [`json`](visualisation/fig5_elec_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_elec_local_local_no_branding.png) [`svg`](visualisation/fig5_elec_local_local_no_branding.svg) [`json`](visualisation/fig5_elec_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_elec_local_no_branding.png) [`svg`](visualisation/fig5_elec_local_no_branding.svg) [`json`](visualisation/fig5_elec_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_elec_local_local_dark.png) [`svg`](visualisation/fig5_elec_local_local_dark.svg) [`json`](visualisation/fig5_elec_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_elec_local_dark.png) [`svg`](visualisation/fig5_elec_local_dark.svg) [`json`](visualisation/fig5_elec_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_elec_local_local_no_branding_dark.png) [`svg`](visualisation/fig5_elec_local_local_no_branding_dark.svg) [`json`](visualisation/fig5_elec_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_elec_local_no_branding_dark.png) [`svg`](visualisation/fig5_elec_local_no_branding_dark.svg) [`json`](visualisation/fig5_elec_local_no_branding_dark.json)   

!["fig5_elec_local"](visualisation/fig5_elec_local.svg "fig5_elec_local")

  

!["fig5_elec_local_dark"](visualisation/fig5_elec_local_dark.svg "fig5_elec_local")

## Figure 6  

Data: [`csv`](data/fig6_lab_local.csv)  
GitHub: [fig6_lab_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/the-phoenix-that-never-rises-whats-holding-back-pakistans-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_lab_local_local.png) [`svg`](visualisation/fig6_lab_local_local.svg) [`json`](visualisation/fig6_lab_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6_lab_local.png) [`svg`](visualisation/fig6_lab_local.svg) [`json`](visualisation/fig6_lab_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_lab_local_local_no_branding.png) [`svg`](visualisation/fig6_lab_local_local_no_branding.svg) [`json`](visualisation/fig6_lab_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6_lab_local_no_branding.png) [`svg`](visualisation/fig6_lab_local_no_branding.svg) [`json`](visualisation/fig6_lab_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_lab_local_local_dark.png) [`svg`](visualisation/fig6_lab_local_local_dark.svg) [`json`](visualisation/fig6_lab_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6_lab_local_dark.png) [`svg`](visualisation/fig6_lab_local_dark.svg) [`json`](visualisation/fig6_lab_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_lab_local_local_no_branding_dark.png) [`svg`](visualisation/fig6_lab_local_local_no_branding_dark.svg) [`json`](visualisation/fig6_lab_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6_lab_local_no_branding_dark.png) [`svg`](visualisation/fig6_lab_local_no_branding_dark.svg) [`json`](visualisation/fig6_lab_local_no_branding_dark.json)   

!["fig6_lab_local"](visualisation/fig6_lab_local.svg "fig6_lab_local")

  

!["fig6_lab_local_dark"](visualisation/fig6_lab_local_dark.svg "fig6_lab_local")

